In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
load_dotenv()

llm= ChatGroq(model = "openai/gpt-oss-120b")
